# Import the libs and base ontologies

In [1]:
from owlready2 import *
import pandas as pd
from neo4j import GraphDatabase, basic_auth
from owlready2 import get_ontology
# from emmopy import get_emmo
from neo4j import GraphDatabase, basic_auth
from sentence_transformers import SentenceTransformer, util
import torch
import re
onto_path.append("")
onto_process=get_ontology("C://Abhishek Thesis//ontology//manufacturing.owl").load()

# onto_matter=get_ontology("C://Abhishek Thesis//ontology//matter.owl").load()
onto_matter=get_ontology("https://raw.githubusercontent.com/IEK-13/MatGraphAI/master/Ontology/matter.owl").load()

onto_quantity=get_ontology("C://Abhishek Thesis//ontology//quantities.owl").load()


* Owlready2 * Warning: optimized Cython parser module 'owlready2_optimized' is not available, defaulting to slower Python implementation


In [2]:
# results = onto_quantity.search_one(label="properties.Quantity")
# print(results)

# Creating node-class 'EMMO_IS_A' relations between exisiting classes.

## map the 3 ontologies on neo4j

In [3]:



# Define Neo4j connection details
uri = "bolt://localhost:7687"
user = "neo4j"
password = "password"

# Create Neo4j driver and session
driver = GraphDatabase.driver(uri, auth=(user, password))
session = driver.session()

# Define function to create nodes
def create_node(tx, label, name,iri):
    tx.run("MERGE (n:" + label + " {name: $name, uri:$uri})", name=name, uri=iri)
    

# Define function to create EMMO_IS_A relationships
def create_emmo_is_a_rel(tx,class_, subclass, superclass):
    tx.run("MATCH (c1:" + class_ + " {name: $subclass}), (c2:" + class_ + " {name: $superclass}) "
           "MERGE (c1)-[:EMMO_IS_A]->(c2)",class_=class_ ,subclass=subclass, superclass=superclass)

# Create nodes for each class in Matter ontology
with session.begin_transaction() as tx:
    for clazz in onto_matter.classes():
        create_node(tx, "EMMOMatter", clazz.name,clazz.iri)

# Create nodes for each class in Process ontology
with session.begin_transaction() as tx:
    for clazz in onto_process.classes():
        create_node(tx, "EMMOProcess", clazz.name,clazz.iri)

# Create nodes for each class in Quantity ontology
with session.begin_transaction() as tx:
    for clazz in onto_quantity.classes():
        create_node(tx, "EMMOQuantity", clazz.name,clazz.iri)

# Create EMMO_IS_A relationships for each class
with session.begin_transaction() as tx:
    for clazz in onto_matter.classes():
        for parent in clazz.is_a:
            create_emmo_is_a_rel(tx,'EMMOMatter', clazz.name, parent.name)
    for clazz in onto_process.classes():
        for parent in clazz.is_a:
            create_emmo_is_a_rel(tx,'EMMOProcess', clazz.name, parent.name)
    for clazz in onto_quantity.classes():
        for parent in clazz.is_a:
            create_emmo_is_a_rel(tx,'EMMOQuantity', clazz.name, parent.name)

# Close Neo4j session and driver
session.close()
driver.close()


# Neo4j codes

In [4]:
# Match(n) optional match(n)-[r]-() delete n,r

#to search for a node
# MATCH (n:Person)
# WHERE n.name = 'John'
# RETURN n


# MATCH (superclass:EMMOMatter)
# WHERE superclass.name = 'Device'
#     return superclass

# Get the Organizational Data from file

In [5]:

# query = f"MATCH (m:{node1} {{name: '{domain_cls}'}}), (ma:{node2} {{name: '{range_cls}'}}) CREATE (m)-[:{rel_type}]->(ma)"
#             result = session.run(query)

In [6]:


# Read the Excel file
df = pd.read_excel('C://Abhishek Thesis//OneDrive_1_5-11-2023//test file-DataCatalog//DataCatalog.xlsx', header=None)

# Find the row index where 'Organizational Data' is located
start_row = df[df.iloc[:, 0] == 'Organizational Data'].index[0]

# Find the row index where 'Characterization Data' is located
end_row = df[df.iloc[:, 0] == 'Characterization Data'].index[0]

# Extract the organizational data rows
organizational_data = df.iloc[start_row + 1: end_row]

# Remove trailing spaces from attribute names
organizational_data.loc[:, 0] = organizational_data.loc[:, 0].str.strip()

# Extract attributes and values
attributes = organizational_data.iloc[:, 0].tolist()
values = organizational_data.iloc[:, 1].tolist()

# Create a dictionary of attributes and values
data_dict = dict(zip(attributes, values))

# Generate variable names dynamically and assign values
for attribute, value in zip(attributes, values):
    var_name = attribute.lower().replace(' ', '_').replace('-', '').replace(':', '')
    globals()[var_name] = value

# Print the variables to verify they contain the expected values
for attribute, value in zip(attributes, values):
    var_name = attribute.lower().replace(' ', '_').replace('-', '').replace(':', '')
    print(f"{var_name}: {globals()[var_name]}")


experiment_title: Elucidating the Influence of the d-Band Center on the Synthesis of Isobutanol
experimentid: 000000-1
measurementid: 000000-1-2
upload_date: 2022-10-11 00:00:00
measurement_date: 2022-10-12 00:00:00
institution: FZJ IEK-13,FZJ IEK-14
founding_body: HIP, ABBC
country: USA,JAPAN
authors_list: Peter,koko
orcid: HJGDF78,uibu5
access_conditions: public
published: https://doi.org/10.3390/catal11030406
topic: Fuel Cells
device: Fuel Cells
component: MembraneElectrodeAssembly
subcomponent: Catalyst Layer
material: Carbon felt electrode
granularity_level: Nanostructure


C:\Users\a.mathur\AppData\Local\Temp\ipykernel_21036\3031692742.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  organizational_data.loc[:, 0] = organizational_data.loc[:, 0].str.strip()


# Load file data to the neo4j ontology and make relationships

In [7]:
# def create_or_get_node(tx, label, **properties):
#     """ check for existing nodes, if not there, create new"""
#     query = "MERGE (n:%s {name: $name}) RETURN n" % label
#     result = tx.run(query, name=properties['name'])
#     if not result.peek():
#         create_node(tx, label, **properties)
     
    
    
def create_nodes(tx, label, **properties):
    query = f"CREATE (n:{label}) SET n += $properties RETURN n"
    return tx.run(query, properties=properties)

def create_or_get_node(tx, label, match_property, properties):
    query = f"MERGE (n:{label} {{{match_property}: $value}}) SET n += $properties RETURN n"
    result = tx.run(query, value=properties[match_property], properties=properties)
    if not result.peek():
        create_nodes(tx, label, **properties)
        
def create_relation(tx, node1, node2, relation, property1, property2, label1, label2):
    query = f"MATCH (n1:{label1}), (n2:{label2}) WHERE n1.{property1} = $name1 AND n2.{property2} = $name2 " \
            f"MERGE (n1)-[r:{relation}]->(n2) " \
            f"RETURN count(r)"
    result = tx.run(query, name1=node1, name2=node2)
    count = result.single()[0]
    return count

# Function to create the relationship if it doesn't exist
# def create_relation(tx, node1, node2, relation, property1, property2):
#     query = f"MATCH (n1), (n2) WHERE n1.{property1} = $name1 AND n2.{property2} = $name2 " \
#             f"MERGE (n1)-[r:{relation}]->(n2) " \
#             f"RETURN count(r)"
#     result = tx.run(query, name1=node1, name2=node2)
#     count = result.single()[0]
#     return count



    
founding_body_list = founding_body.split(",")    
country_list = country.split(",")
institution_list = institution.split(",")
orcid_list = orcid.split(",")
authors_list_list = authors_list.split(",")

# Create nodes for Researcher, Institution and Country
with driver.session() as session:
    with session.begin_transaction() as tx:
    
        for country_ in country_list:
            if country not in ('','-','nan'):
                properties = {'name': country_}
                create_or_get_node(tx, "Country",'name', properties)
                
        for founding_body_ in founding_body_list:
            if founding_body_ not in ('','-','nan'):
                properties = {'name': founding_body_}
                create_or_get_node(tx, "FundingBody",'name', properties)
        
    
        for institution_ in institution_list:
            if institution not in ('','-','nan'):
                properties = {'name': institution_}
                create_or_get_node(tx, "Institution",'name', properties)

        for authors_list_,orcid_ in zip(authors_list_list,orcid_list):
            if orcid not in ('', '-', 'nan'):
                properties = {'name': authors_list_, 'orcid': orcid_}
                create_or_get_node(tx, "Researcher", 'orcid', properties)

        
#         if orcid not in ('','-','nan'): 
#             create_or_get_node(tx, "Researcher", name=authors_list, orcid=orcid)
        
                
        
        if subcomponent not in ('','-','nan'):    
            properties = {'name': subcomponent}
            create_or_get_node(tx, "Subcomponent",'name', properties)

        
#         if access_conditions not in ('','-','nan'):    
#             properties = {'name': access_conditions}
#             create_or_get_node(tx, "Parameter",'name', properties)

        
                
        if component not in ('','-','nan'):    
            properties = {'name': component}
            create_or_get_node(tx, "Component",'name', properties)
  
        if device not in ('','-','nan'): 
            properties = {'name': device}
            create_or_get_node(tx, "Device",'name', properties)

        

        if material not in ('','-','nan'):  
            properties = {'name': material}
            create_or_get_node(tx, "Material", 'name',properties)

                     

        properties = {'name': measurementid,'experiment_start':measurement_date,
                    'date_added':upload_date, 'run_title' :experiment_title, 'experiment_id':experimentid, 'access_conditions':access_conditions}
        create_or_get_node(tx, "Measurement",'name',properties)

        
        
# config_rel_lst=[{'node1':orcid, 'node2':institution,'relation': 'AFFILIATED_TO', 'property_name1':'orcid','property_name2': 'name'},
#       {'node1':institution, 'node2':country,'relation': 'IN', 'property_name1':'name','property_name2': 'name'},
#       {'node1':orcid, 'node2':country,'relation': 'IN', 'property_name1':'orcid','property_name2': 'name'},
#       {'node1':measurement_id, 'node2':orcid,'relation': 'BY', 'property_name1':'name','property_name2': 'orcid'} 
#      ]
           
# Create relations between the nodes  
    with session.begin_transaction() as tx:  
    # Check and create relationship
        for orcid_,institution_  in zip(orcid_list,institution_list):
            count = create_relation(tx, orcid_, institution_, 'AFFILIATED_TO', 'orcid', 'name','Researcher', 'Institution')

            if count == 0:
#                 print("Relationship AFFILIATED_TO doesn't exist, creating it...")
                create_relation(tx, orcid_, institution_, 'AFFILIATED_TO', 'orcid', 'name','Researcher', 'Institution')
            else:
                pass

        for institution_, country_ in zip(institution_list,country_list):

            count = create_relation(tx, institution_, country_, 'IN', 'name', 'name','Institution', 'Country')

            if count == 0:
#                 print("Relationship IN doesn't exist, creating it...")
                create_relation(tx, institution_, country_, 'IN', 'name', 'name','Institution', 'Country')
            else:
                pass
        
        
        for orcid_, country_ in zip(orcid_list,country_list):
            count = create_relation(tx, orcid_, country_, 'IN', 'orcid', 'name','Researcher', 'Country')

            if count == 0:
#                 print("Relationship IN_ orcid doesn't exist, creating it...")
                create_relation(tx, orcid_, country_, 'IN', 'orcid', 'name','Researcher', 'Country')
            else:
                pass
        
        
        for orcid_ in orcid_list:
            count = create_relation(tx, measurementid, orcid_, 'BY', 'name', 'orcid','Measurement', 'Researcher')

            if count == 0:
#                 print("Relationship BY doesn't exist, creating it...")
                create_relation(tx, measurementid, orcid_, 'BY', 'name', 'orcid','Measurement', 'Researcher')
            else:
                pass
        
        
        count = create_relation(tx, material, measurementid, 'IS_MEASUREMENT_INPUT', 'name', 'name','Material', 'Measurement')
        if count == 0:
#                 print("Relationship IN_ orcid doesn't exist, creating it...")
            create_relation(tx,material, measurementid, 'IS_MEASUREMENT_INPUT', 'name', 'name','Material', 'Measurement')
        else:
            pass
        
      
        count = create_relation(tx, component, subcomponent, 'HAS_PART', 'name', 'name','Component', 'Subcomponent')
        if count == 0:
#                 print("Relationship IN_ orcid doesn't exist, creating it...")
            create_relation(tx,component, subcomponent, 'HAS_PART', 'name', 'name','Component', 'Subcomponent')
        else:
            pass
            
            
        count = create_relation(tx, device, component, 'HAS_PART', 'name', 'name','Device', 'Component')
        if count == 0:
#                 print("Relationship IN_ orcid doesn't exist, creating it...")
            create_relation(tx,  device, component, 'HAS_PART', 'name', 'name','Device', 'Component')
        else:
            pass

        
        count = create_relation(tx, component, material, 'HAS_PART', 'name', 'name','Component', 'Material')
        if count == 0:
#                 print("Relationship IN_ orcid doesn't exist, creating it...")
            create_relation(tx,  component, material, 'HAS_PART', 'name', 'name','Component', 'Material')
        else:
            pass

        count = create_relation(tx, component, measurementid, 'IS_MEASUREMENT_INPUT', 'name', 'name','Component', 'Measurement')
        if count == 0:
#                 print("Relationship IN_ orcid doesn't exist, creating it...")
            create_relation(tx,  component, measurementid, 'IS_MEASUREMENT_INPUT', 'name', 'name','Component', 'Measurement')
        else:
            pass
            
          
        for founding_body_ in founding_body_list:
            count = create_relation(tx,measurementid , founding_body_,  'FUNDED_BY', 'name', 'name','Measurement', 'FundingBody')
        if count == 0:
#                 print("Relationship BY doesn't exist, creating it...")
            create_relation(tx, measurementid , founding_body_,  'FUNDED_BY', 'name', 'name','Measurement', 'FundingBody')
        else:
            pass
        
        
#         create_relation4(tx, component, 'Imaging', 'new_rel', 'name', 'name', 'EMMOMatter', 'EMMOProcess')

# To get the classes and subclasses from ontology, so as to create further IS_A relations with file data

In [8]:
from neo4j import GraphDatabase


def get_all_subclasses(driver, target_class_name,node_lab):
    with driver.session() as session:
#        result = session.run(
#            "MATCH (target {name: $targetClassName})<-[:$relationshipType*]-(subclass: {name: $targetClassName}) RETURN DISTINCT subclass.name",
#                    targetClassName=target_class_name,
#                    relationshipType=relationship_type)
        
        result = session.run(
            "MATCH (target:{node_lab} {{name: $targetClassName}})<-[:EMMO_IS_A*]-(subclass:{node_lab}) "
            "RETURN DISTINCT subclass.name".format(node_lab=node_lab),
            targetClassName=target_class_name
        )
        return set([record["subclass.name"] for record in result])




In [9]:
# Example usage
target_class_name = "Component"
node_lab='EMMOMatter'
# Create a Neo4j driver instance
driver = GraphDatabase.driver("bolt://localhost:7687", auth=("neo4j", "password"))

# Call the function to retrieve all subclasses
all_subclasses = get_all_subclasses(driver, target_class_name,node_lab)

# Include the target class itself in the set of subclasses
all_subclasses.add(target_class_name)

# Convert the set to a list
component_lst = list(all_subclasses)

# Print the subclasses
print(component_lst)

['Electrode', 'PerfluorinatedMembrane', 'ConversionElectrode', 'MembraneElectrodeAssembly', 'SimpleElectrode', 'PlatinumElectrode', 'IntercalationElectrode', 'Flemion', 'NRE211', 'NegativeElectrode', 'InertElectrode', 'CounterElectrode', 'RotationDiskElectrode', 'Component', 'IndicatorElectrode', 'Membrane', 'AquivionSO3H-Form', 'CatalystCoatedMembrane', 'PartiallyFluorinatedMembrane', 'PositiveElectrode', '5LayerMEA', 'Asiplex', 'HydrogenElectrode', 'CompositeElectrode', 'NonFluorinatedCompositeMembrane', 'MetalElectrode', 'GasDiffusionElectrode', 'NafionMembrane', 'NonPolarizableElectrode', 'Gore-Select', 'ReferenceElectrode', 'WorkingElectrode', 'CCM', 'CatalystLayer', 'NonFluorinatedMembrane', 'NormalHydrogenElectrode', 'Cathode', 'AquivionD98-25BS', 'NRE212', 'AquivionD79-25BS', 'CarbonPasterElectrode', 'ElectrochemicalComponent', 'LithiumIntercalationElectrode', 'Aquivion', 'PolarizableElectrode', 'PorousElectrode', 'Anode', 'FuelCellComponent', 'AquivionD72-25BS', 'Neosepta-F', 

In [10]:
target_class_name = "Device"
node_lab='EMMOMatter'
driver = GraphDatabase.driver("bolt://localhost:7687", auth=("neo4j", "password"))
all_subclasses = get_all_subclasses(driver, target_class_name,node_lab)
all_subclasses.add(target_class_name)
device_lst = list(all_subclasses)
print(component_lst)

['Electrode', 'PerfluorinatedMembrane', 'ConversionElectrode', 'MembraneElectrodeAssembly', 'SimpleElectrode', 'PlatinumElectrode', 'IntercalationElectrode', 'Flemion', 'NRE211', 'NegativeElectrode', 'InertElectrode', 'CounterElectrode', 'RotationDiskElectrode', 'Component', 'IndicatorElectrode', 'Membrane', 'AquivionSO3H-Form', 'CatalystCoatedMembrane', 'PartiallyFluorinatedMembrane', 'PositiveElectrode', '5LayerMEA', 'Asiplex', 'HydrogenElectrode', 'CompositeElectrode', 'NonFluorinatedCompositeMembrane', 'MetalElectrode', 'GasDiffusionElectrode', 'NafionMembrane', 'NonPolarizableElectrode', 'Gore-Select', 'ReferenceElectrode', 'WorkingElectrode', 'CCM', 'CatalystLayer', 'NonFluorinatedMembrane', 'NormalHydrogenElectrode', 'Cathode', 'AquivionD98-25BS', 'NRE212', 'AquivionD79-25BS', 'CarbonPasterElectrode', 'ElectrochemicalComponent', 'LithiumIntercalationElectrode', 'Aquivion', 'PolarizableElectrode', 'PorousElectrode', 'Anode', 'FuelCellComponent', 'AquivionD72-25BS', 'Neosepta-F', 

In [11]:
target_class_name = "Subcomponent"
node_lab='EMMOMatter'
driver = GraphDatabase.driver("bolt://localhost:7687", auth=("neo4j", "password"))
all_subclasses = get_all_subclasses(driver, target_class_name,node_lab)
all_subclasses.add(target_class_name)
subcomponent_lst = list(all_subclasses)
print(subcomponent_lst)

['CompositeBipolarPlate', 'CoolingPlate', 'FuelCellSubcomponent', 'StainlessSteelBasedNobleMetalCoatedBipolarPlate', 'TitaniumBasedDiamondLikeCarbonCoatedBipolarPlate', 'StainlessSteelBasedMetalNitrideCoatedBiploarPlate', 'TitaniumBasedMtealNitrideCoatedBipolarPlate', 'StructuralSubcomponent', 'MetalBasedCompositeBipolarPlate', 'ClothGDL', 'AluminiumBasedMetalBasedCoatedBipolarPlate', 'NickelBasedGraphiteCoatedBipolarPlate', 'StainlessSteelBasedMetalBasedCoatedBipolarPlate', 'StainlessSteelBasedCarbonBasedCoatedBipolarPlate', 'NickelBasedBipolarPlate', 'TitaniumBasedMetalCarbideCoatedBipolarPlate', 'NickelBasedMetalNitrideCoatedBipolarPlate', 'TitaniumBasedOrganicSelfAssembleMonoPolymerCoatedBipolarPlate', 'NickelBasedMetalCarbideCoatedBipolarPlate', 'FeltGDL', 'SwagelokCellHousing', 'TitaniumBasedConductivePolymerCoatedBipolarPlate', 'Binder', 'StainlessSteelBasedOrganicSelfAssembleMonoPolymerCoatedBipolarPlate', 'PrismaticCellHousing', 'CoatedMetallicBipolarPlate', 'NickelBasedNobleM

In [12]:
target_class_name = "Material"
node_lab='EMMOMatter'
driver = GraphDatabase.driver("bolt://localhost:7687", auth=("neo4j", "password"))
all_subclasses = get_all_subclasses(driver, target_class_name,node_lab)
all_subclasses.add(target_class_name)
material_lst = list(all_subclasses)
print(material_lst)

['IsotropicMaterial', 'EA30E', 'DecalSusbtrate', 'F30E-HT', 'NearNeutralElectrolyte', '2DNanoMaterial', 'Ionomer', 'Copolymer', 'ConductingPolymerSupportedPlatinumCatalyst', 'Gore', 'Material', 'EnergyMaterial', 'CoatingSubstrate', 'BinaryCarbonSupportedPlatinumCatalyst', 'IridiumOxideCatalyst', 'PTFESubstrate', 'PlatinumCatalyst', '800EWIonomer', 'F50E', 'Polymer', '0DNanoMaterial', 'HomoGenousMaterial', 'Polyepoxide', 'NanoMaterial', 'IntercalationMaterial', 'BinaryMaterial', 'ETFE', 'Catholyte', 'ManufacturedMaterial', 'NaturalMaterial', 'AnisotropicMaterial', 'AcetyleneBlack', 'LiquidElectrolyte', 'AlkalineElectrolyte', 'LampBlack', 'ElectroCatalyst', 'EA30E-HT', 'SecundaryCatalyst', 'TernaryCatalyst', 'Nafion_D521CS', '1DNanoMaterial', 'BinaryCatalyst', 'AquivionD72-25BS', 'Nafion_D2021CS', 'ElectrolyteSolution', 'CarbonBlack', 'IonicLiquidElectrolyte', '3DNanoMaterial', 'ActiveMaterial', 'FunctionalMaterial', 'ElectroChemicalIntercalationMaterial', 'ChannelBlack', 'Nafion_D2020CS

In [13]:
from neo4j import GraphDatabase

def create_subclass(driver, node_label, superclass_name, subclass_name, url):
    with driver.session() as session:
        result = session.run(
            "MATCH (superclass:" + node_label + ") "
            "WHERE superclass.name = $superclassName "
            "MERGE (subclass:EMMOMatter { name: $subclassName }) "
            "ON CREATE SET subclass.url = $url "
            "MERGE (subclass)-[:EMMO_IS_A]->(superclass) "
            "RETURN subclass, superclass",
            superclassName=superclass_name,
            subclassName=subclass_name,
            url=url + subclass_name  # Append subclass_name to the URL
        )
        return result.single()




# Creation of IS_A relationship of meta data (organizational data) to EMMO classes.

In [14]:
from sentence_transformers import SentenceTransformer, util
"""
This is a simple application for sentence embeddings: semantic search

We have a corpus with various sentences. Then, for a given query sentence,
we want to find the most similar sentence in this corpus.

This script outputs for various queries the top 3 most similar sentences in the corpus.
"""

def modify_strings(string):
    modified_lst=''
   

    if len(string.split())==1:
        print('string is ',string)
        if string.isupper()==True:         
            modified_lst=string
            
        elif string[0].isupper()==True and string[1].isupper()==False:
            modified_lst=string
#                 print(modified_string)

        else:
            modified_lst=string.title()

    else:     

        modified_string = ''.join(word.capitalize() for word in string.split())
        modified_lst=modified_string
    return modified_lst



def emmo_cls_onto(queries,emmo_lst,node_label,superclass_name,url):
    
    embedder = SentenceTransformer('all-MiniLM-L6-v2')

    # Corpus with example sentences
    cls_lst_onto = emmo_lst
    cls_str_onto = []
    for n in cls_lst_onto:
        cls_str_onto.append(str(n))
    #print(abc_str)


    corpus = cls_str_onto
    corpus_embeddings = embedder.encode(corpus, convert_to_tensor=True)
    cls_dict ={}
    # Query sentences:
    # queries = class_list
    # Find the closest 3 sentences of the corpus for each query sentence based on cosine similarity
    top_k = min(3, len(corpus))

    subclass_name = queries
#         words = subclass_name.split()
#         capitalized_words = [word.capitalize() for word in words]
#         query = ''.join(capitalized_words)
    query = modify_strings(subclass_name)
    query_embedding = embedder.encode(query, convert_to_tensor=True)

    c_lst=[]
    scr_lst=[]

    # We use cosine-similarity and torch.topk to find the highest 3 scores
    cos_scores = util.cos_sim(query_embedding, corpus_embeddings)[0]
    top_results = torch.topk(cos_scores, k=top_k)
#     print(top_results)
    print("\n\n======================\n\n")
    print("Query:", query)
    print("\nTop 3 most similar sentences in corpus:")

    for score, idx in zip(top_results[0], top_results[1]):
        print('c_lst ',corpus[idx], "(Score: {:.4f})".format(score))
        c_lst.append(corpus[idx].replace('mason.', ''))
        scr_lst.append("{:.4f}".format(score)) 
#         print('c_lst ',c_lst)


    with driver.session() as session:
        with session.begin_transaction() as tx:

            if float(scr_lst[0])>=0.9:
                spr_cls = c_lst[0]  

                create_relation(tx,subclass_name ,spr_cls, "IS_A", 'name', 'name',superclass_name,node_label)
                print(query,c_lst[0],superclass_name)

#                 with session.begin_transaction() as tx:   
#                     if device not in ('','-','nan'):  
#                         properties = {'name': material}
#                         create_or_get_node(tx, "Material", properties)


            elif float(scr_lst[0])<0.6:     
                # Example usage

                # Create a Neo4j driver instance
                # Call the function to create the subclass and "IS_A" relationship
                create_subclass(driver, node_label, superclass_name, query, url)
                create_relation(tx, subclass_name ,query, "IS_A",  'name', 'name',superclass_name,node_label)

                print('outside func ',superclass_name ,node_label, query, url)
#            createNewClass(query,spr_cls)

In [15]:
node_label = 'EMMOMatter'
superclass_name='Device'
url = 'http://www.semanticweb.org/maxdreger/ontologies/2022/10/matter/#'
# d_lst=['Catalysis','Lithium Battery','PEMFC','Fuel Cells','FUELCELLS','FuelCells','Vanadium redox flow battery','VanadiumRedoxFlowBattery','Proton Exchange Membrane Fuel Cell','LithiumBattery','Polymer electrolyte fuel cell','PolymerElectrolyteFuelCell']
cls_dict=emmo_cls_onto(device,device_lst,node_label,superclass_name,url)






Query: FuelCells

Top 3 most similar sentences in corpus:
c_lst  FuelCell (Score: 0.9472)
c_lst  FuelCellStack (Score: 0.8775)
c_lst  BatteryCell (Score: 0.6030)
FuelCells FuelCell Device


In [16]:
# with driver.session() as session:
#     with session.begin_transaction() as tx:
#         node_label = 'EMMOMatter'
#         superclass_name='Device'
#         spr_cls='FuelCell' 
#         query='FuelCells'
#         create_relation(tx,query ,spr_cls, "IS_A", 'name', 'name')

In [17]:
node_label = 'EMMOMatter'
superclass_name='Component'
url = 'http://www.semanticweb.org/maxdreger/ontologies/2022/10/matter/#'
# c_lst_=['Catalyst layer','Membrane electrode assembly','MEA','Battery Electrode']
cls_dict=emmo_cls_onto(component,component_lst,node_label,superclass_name,url)


string is  MembraneElectrodeAssembly




Query: MembraneElectrodeAssembly

Top 3 most similar sentences in corpus:
c_lst  MembraneElectrodeAssembly (Score: 1.0000)
c_lst  Membrane (Score: 0.6848)
c_lst  HydrogenElectrode (Score: 0.5968)
MembraneElectrodeAssembly MembraneElectrodeAssembly Component


In [18]:
node_label = 'EMMOMatter'
superclass_name='Subcomponent'
url = 'http://www.semanticweb.org/maxdreger/ontologies/2022/10/matter/#'
# s_lst_=['Catalyst','Gas diffusion electrode','Catalyst Layer']
cls_dict=emmo_cls_onto(subcomponent,subcomponent_lst,node_label,superclass_name,url)







Query: CatalystLayer

Top 3 most similar sentences in corpus:
c_lst  GasDiffusionLayer (Score: 0.4745)
c_lst  TitaniumBasedDiamondLikeCarbonCoatedBipolarPlate (Score: 0.4340)
c_lst  TitaniumBasedGraohiteCoatedBipolarPlate (Score: 0.4150)
outside func  Subcomponent EMMOMatter CatalystLayer http://www.semanticweb.org/maxdreger/ontologies/2022/10/matter/#


In [19]:
node_label = 'EMMOMatter'
superclass_name='Material'
url = 'http://www.semanticweb.org/maxdreger/ontologies/2022/10/matter/#'
# lst_=['30 wt% Pd/C','5 wt% Pd/C','Gas diffusion layer','0.2 mg/cm2 Pt MEA','platinum','Carbon felt electrode','NMC111 electrode','LixMnO2','Pt']
cls_dict=emmo_cls_onto(material,material_lst,node_label,superclass_name,url)





Query: CarbonFeltElectrode

Top 3 most similar sentences in corpus:
c_lst  CarbonSupportedPlatinumCatalyst (Score: 0.4882)
c_lst  ManufacturedMaterial (Score: 0.4380)
c_lst  CarbonBlack (Score: 0.4215)
outside func  Material EMMOMatter CarbonFeltElectrode http://www.semanticweb.org/maxdreger/ontologies/2022/10/matter/#


In [20]:
# print(Measurement_ID,Experiment_ID=Experiment_ID, Experiment_Title=Experiment_Title, Measurement_date=Measurement_date)

# Get the Characterization Data from file

In [21]:
# Find the row index where 'Organizational Data' is located
start_row = df[df.iloc[:, 0] == 'Characterization Data'].index[0]

# Find the row index where 'Characterization Data' is located
end_row = df[df.iloc[:, 0] == 'Sample preparation'].index[0]

# Extract the organizational data rows
Characterization_measurement_data = df.iloc[start_row + 1: end_row]

# Remove trailing spaces from attribute names
Characterization_measurement_data.loc[:, 0] = Characterization_measurement_data.loc[:, 0].str.strip()

# Extract attributes and values
attributes = Characterization_measurement_data.iloc[:, 0].tolist()
values = Characterization_measurement_data.iloc[:, 1].tolist()

# Create a dictionary of attributes and values
data_dict = dict(zip(attributes, values))

# Generate variable names dynamically and assign values
for attribute, value in zip(attributes, values):
    var_name = attribute.lower().replace(' ', '_').replace('-', '').replace(':', '')
    globals()[var_name] = value

# Print the variables to verify they contain the expected values
for attribute, value in zip(attributes, values):
    var_name = attribute.lower().replace(' ', '_').replace('-', '').replace(':', '')
    print(f"{var_name}: {globals()[var_name]}")


measurement_type: SEM
specimen: homogeneous powder


C:\Users\a.mathur\AppData\Local\Temp\ipykernel_21036\3654495869.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Characterization_measurement_data.loc[:, 0] = Characterization_measurement_data.loc[:, 0].str.strip()


In [22]:
# Find the row index where 'Organizational Data' is located
excel_file = 'C://Abhishek Thesis//OneDrive_1_5-11-2023//test file-DataCatalog//DataCatalog.xlsx'
df = pd.read_excel(excel_file, header=None)

# Find the row index where 'Sample preparation' is located
# Find the row index where 'Sample preparation' is located
start_row = 'Sample preparation'
end_row = 'Characterization environment'

start_index = df[df.iloc[:, 0] == start_row].index[0]
end_index = df[df.iloc[:, 0] == end_row].index[0]

# Extract the desired table based on the row indices
table_data = df.iloc[start_index+1:end_index]

# Assign the first row as column names
table_data.columns = table_data.iloc[0]
table_data = table_data.iloc[1:].reset_index(drop=True)

# Remove the unwanted numerical column
table_data = table_data.loc[:, ~table_data.columns.astype(str).str.isnumeric()]
table_data


23,Step,Precursor,Amount,Technique,Condition,Target,Amount,NaN
0,1,Carbon felt electrode,12.5 mm,SamplePreparation,24 hour,F30E,5 mL,NaN
1,2,"F30E, LiquidElectrolyte",2,CCLManufacturing,"50 °C, 10 min, 250 pA, 5 keV",PTFESubstrate,6 ml,NaN
2,3,"PTFESubstrate, ElectroChemicalIntercalationMat...",3 mm,5LayerMEAManufacturing,24 hour,Polyepoxide,6 ml,NaN
3,4,Polyepoxide,2,FuelCellAssembly,34 min,Anolyte,6 ml,NaN
4,5,"Anolyte, CarbonSupportedPlatinumCatalyst, Furn...",30 min,AdditiveManufacturing,12 mm,Sample,7 ml,NaN


In [23]:
# Loop through each row and print the column values
for index, row in table_data.iterrows():
    print(f"Row {index+1}:")
    for column, value in row.items():
        if not pd.isna(value):
            print(f"{column}: {value}")
    print()

Row 1:
Step: 1
Precursor: Carbon felt electrode
Amount: 12.5 mm
Technique: SamplePreparation
Condition: 24 hour
Target: F30E
Amount: 5 mL

Row 2:
Step: 2
Precursor: F30E, LiquidElectrolyte
Amount: 2
Technique: CCLManufacturing
Condition: 50 °C, 10 min, 250 pA, 5 keV
Target: PTFESubstrate
Amount: 6 ml

Row 3:
Step: 3
Precursor: PTFESubstrate, ElectroChemicalIntercalationMaterial
Amount: 3 mm
Technique: 5LayerMEAManufacturing
Condition: 24 hour
Target: Polyepoxide
Amount: 6 ml

Row 4:
Step: 4
Precursor: Polyepoxide
Amount: 2
Technique: FuelCellAssembly
Condition: 34 min
Target: Anolyte
Amount: 6 ml

Row 5:
Step: 5
Precursor: Anolyte, CarbonSupportedPlatinumCatalyst, Furnace Black
Amount: 30 min
Technique: AdditiveManufacturing
Condition: 12 mm
Target: Sample
Amount: 7 ml



In [24]:
with driver.session() as session:
    with session.begin_transaction() as tx:
        properties = {'name': "SamplePreparation_"+measurementid}
        create_or_get_node(tx, "SamplePreparation",'name', properties)

In [25]:
def create_SamplePrepsubclass(tx, node_label, superclass_name, subclass_name):
    result = tx.run(
        "MATCH (superclass:" + node_label + ") "
        "WHERE superclass.name = $superclassName "
        "CREATE (subclass:" + node_label + " {name: $subclassName})"
        "MERGE (subclass)-[:IS_A]->(superclass) "
        "RETURN subclass",
        superclassName=superclass_name,
        subclassName=subclass_name
    )
    return result.single()




In [26]:
# Usage example:
with driver.session() as session:
    h=0
    for index, row in table_data.iterrows():
        step_number = row['Step']
        h=h+1
        tx = session.begin_transaction()
        create_SamplePrepsubclass(tx, "SamplePreparation", "SamplePreparation_"+measurementid, step_number)
        tx.commit()
    

In [27]:

with driver.session() as session:
    with session.begin_transaction() as tx:
        for i in range(1, h):
            create_relation(tx, i, i + 1, "FOLLOWED_BY", 'name', 'name', 'SamplePreparation', 'SamplePreparation')
        tx.commit()